In [ ]:
# Setup for Google Colab
import sys
import subprocess
import os

# Install LightGlue and dependencies if on Colab
if 'google.colab' in sys.modules:
    print("Running on Google Colab - Installing dependencies...")
    
    # Clone the repository to get reference data
    repo_url = "https://github.com/marcusleiwe/sea-turtle_facial-recognition.git"
    if not os.path.exists("sea-turtle_facial-recognition"):
        print("Cloning repository...")
        subprocess.run(["git", "clone", repo_url], check=True)
        # Change to the project directory
        os.chdir("sea-turtle_facial-recognition")
        print("Repository cloned successfully!")
    else:
        os.chdir("sea-turtle_facial-recognition")
        print("Repository already exists, using existing copy")
    
    # Install dependencies
    subprocess.run(["pip", "install", "git+https://github.com/cvg/LightGlue.git"], check=True)
    subprocess.run(["pip", "install", "opencv-python-headless"], check=True)
    print("Dependencies installed!")
else:
    print("Running locally - assuming dependencies are installed via Poetry/Conda")

In [ ]:
# Import required libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import glob
from pathlib import Path
import time
import torch
from sklearn.metrics import auc

# LightGlue imports
from lightglue import LightGlue, SuperPoint, SIFT
from lightglue.utils import load_image, rbd

# Set style for visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

print("All libraries imported successfully!")

## 1. Load Reference Library and Query Images

First, we'll load the pre-built reference library and our query images for identification.

In [ ]:
# Load reference library
library_path = "reference_library.pkl"
try:
    with open(library_path, 'rb') as f:
        reference_library = pickle.load(f)
    
    print(f"✅ Reference library loaded successfully!")
    print(f"   📊 Contains {len(reference_library)} turtle images:")
    for name in sorted(reference_library.keys()):
        num_kp = reference_library[name]['num_keypoints']
        print(f"      - {name}: {num_kp} keypoints")
        
except FileNotFoundError:
    print(f"❌ Reference library not found at {library_path}")
    print("   Please run the reference library creation notebook first!")
    reference_library = None

In [ ]:
# Check reference library structure
print("🔍 REFERENCE LIBRARY STRUCTURE ANALYSIS")
print("=" * 50)

# Pick one example to examine
sample_name = list(reference_library.keys())[0]
sample_data = reference_library[sample_name]

print(f"Sample turtle: {sample_name}")
print(f"Keys in sample data: {list(sample_data.keys())}")

for key, value in sample_data.items():
    if isinstance(value, torch.Tensor):
        print(f"  {key}: torch.Tensor, shape={value.shape}, dtype={value.dtype}")
    else:
        print(f"  {key}: {type(value).__name__} = {value}")

print("\n" + "=" * 50)
print("All reference library entries:")
for name, data in reference_library.items():
    if data is not None:
        keys = list(data.keys())
        has_scales = 'scales' in keys
        has_oris = 'oris' in keys
        print(f"  {name:20s} | Keys: {keys} | Has scales: {has_scales} | Has oris: {has_oris}")
    else:
        print(f"  {name:20s} | None (failed extraction)")

In [ ]:
# Load query images from new_samples
query_dir = Path("data/new_samples")
if not query_dir.exists():
    print(f"Warning: {query_dir} not found. Creating directory...")
    query_dir.mkdir(parents=True, exist_ok=True)

query_paths = list(query_dir.glob("*.jpg")) + list(query_dir.glob("*.JPG"))
print(f"\nFound {len(query_paths)} query images:")

query_images = {}
for img_path in query_paths:
    img = cv2.imread(str(img_path))
    if img is not None:
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        query_images[img_path.stem] = img_rgb
        print(f"  - {img_path.name} - Shape: {img_rgb.shape}")

print(f"\nLoaded {len(query_images)} query images for identification")

## 2. Initialize LightGlue Matcher

LightGlue is a learned feature matcher that produces confidence scores for keypoint correspondences.

In [ ]:
# Initialize SIFT extractor and LightGlue matcher with custom configuration
extractor = SIFT(max_num_keypoints=2048).eval().to(device)

# Custom LightGlue configuration optimized for turtle identification
custom_conf = {
    "name": "lightglue",  # just for interfacing
    "input_dim": 256,  # input descriptor dimension (autoselected from weights)
    "descriptor_dim": 256,
    "add_scale_ori": False,
    "n_layers": 9,  # From my memory we usually exit way before all 9 layers are used
    "num_heads": 4,
    "flash": True,  # enable FlashAttention if available.
    "mp": False,  # enable mixed precision
    "depth_confidence": -1,  # DISABLE early stopping for better matches
    "width_confidence": -1,  # DISABLE point pruning for better matches
    "filter_threshold": 0.1,  # match threshold - none of our metrics use this apart from `nr_match` so we don't need to tune this.
    "weights": None,
}

matcher = LightGlue(features='sift', **custom_conf).eval().to(device)

print("LightGlue matcher initialized with custom configuration!")
print(f"  - Feature extractor: SIFT (max 2048 keypoints)")
print(f"  - Matcher: LightGlue with disabled confidence thresholds")
print(f"  - Key changes:")
print(f"    • depth_confidence: -1 (disabled early stopping)")
print(f"    • width_confidence: -1 (disabled point pruning)")
print(f"    • This allows more potential matches to be considered")
print(f"  - Device: {device}")

## 3. Turtle Identification Function

Our novel identification approach uses confidence scores from LightGlue matching to create cumulative distribution plots, then calculates AUC as the final matching score.

In [ ]:
def identify_turtle_fixed(query_image_path, reference_library, extractor, matcher, device, debug_plots=False):
    """
    Turtle identification using YOUR EXACT working AUC methodology with proper (1,1) handling.
    """
    
    print(f"\n🔍 Identifying turtle: {Path(query_image_path).stem}")
    print("=" * 60)
    
    # Load and preprocess query image
    query_tensor = load_image(query_image_path).to(device)
    
    # Extract query features using .extract() method like your working code
    with torch.no_grad():
        query_features = extractor.extract(query_tensor)
    
    # Store results
    match_results = {}
    all_scores = []
    
    start_time = time.time()
    
    # For debug plotting
    if debug_plots:
        fig, axes = plt.subplots(2, 5, figsize=(20, 8))
        axes = axes.flatten()
        plot_idx = 0
    
    # Compare against each reference image
    for ref_name, ref_data in reference_library.items():
        if ref_data is None:
            continue
        
        try:
            # Move reference features to device (only tensor fields)
            ref_features = {k: v.to(device) for k, v in ref_data.items() if isinstance(v, torch.Tensor)}
            
            # Perform matching with LightGlue
            matches = matcher({'image0': query_features, 'image1': ref_features})
            
            # Get matching scores and compute AUC using YOUR exact method
            if 'matching_scores0' in matches and matches['matching_scores0'] is not None:
                matching_scores = matches['matching_scores0']
                
                # Use YOUR EXACT AUC computation with proper (1,1) endpoint
                auc_score = compute_auc_from_scores_torch_exact(matching_scores)
                
                # Convert to numpy for additional analysis
                scores_np = matching_scores.detach().cpu().numpy().flatten()
                valid_scores = scores_np[scores_np > -1]  # Filter like your method
                
                if len(valid_scores) > 0:
                    # Debug plotting using the exact torch method for ECDF with (1,1) endpoint
                    if debug_plots and plot_idx < len(axes):
                        # Recreate the exact ECDF from your torch method WITH (1,1) endpoint
                        valid_torch = matching_scores[matching_scores > -1]
                        sorted_torch = torch.sort(valid_torch)[0]
                        ecdf_torch = torch.linspace(0, 1, steps=sorted_torch.numel(), device=sorted_torch.device)
                        
                        # Add (1,1) endpoint if needed (same as AUC calculation)
                        if sorted_torch[-1] < 1.0:
                            sorted_torch = torch.cat([sorted_torch, torch.tensor([1.0], device=sorted_torch.device)])
                            ecdf_torch = torch.cat([ecdf_torch, torch.tensor([1.0], device=ecdf_torch.device)])
                        
                        # Convert to numpy for plotting
                        x_plot = sorted_torch.detach().cpu().numpy()
                        y_plot = ecdf_torch.detach().cpu().numpy()
                        
                        axes[plot_idx].plot(x_plot, y_plot, 'b-', linewidth=2)
                        axes[plot_idx].fill_between(x_plot, y_plot, alpha=0.3)
                        axes[plot_idx].set_title(f'{ref_name}\nAUC: {auc_score:.4f}\nMatches: {len(valid_scores)}', fontsize=9)
                        axes[plot_idx].set_xlabel('Confidence Score')
                        axes[plot_idx].set_ylabel('ECDF')
                        axes[plot_idx].grid(True, alpha=0.3)
                        axes[plot_idx].set_xlim([0, 1])
                        axes[plot_idx].set_ylim([0, 1])
                        plot_idx += 1
                    
                    match_results[ref_name] = {
                        'auc_score': auc_score,
                        'num_matches': len(valid_scores),
                        'total_keypoints': len(scores_np),
                        'avg_confidence': np.mean(valid_scores),
                        'max_confidence': np.max(valid_scores),
                        'confidence_scores': valid_scores
                    }
                    
                    all_scores.append(auc_score)
                    
                    print(f"{ref_name:20s} | AUC: {auc_score:.4f} | Matches: {len(valid_scores):3d}/{len(scores_np):3d} | Avg: {np.mean(valid_scores):.3f} | Max: {np.max(valid_scores):.3f}")
                    
                else:
                    print(f"{ref_name:20s} | No valid matches")
                    match_results[ref_name] = {
                        'auc_score': 0.0,  # Use 0.0 like your method for no matches
                        'num_matches': 0,
                        'total_keypoints': len(scores_np),
                        'avg_confidence': 0.0,
                        'max_confidence': 0.0,
                        'confidence_scores': np.array([])
                    }
                    all_scores.append(0.0)
            
            else:
                print(f"{ref_name:20s} | No matching_scores0 in matches")
                match_results[ref_name] = {
                    'auc_score': 0.0,
                    'num_matches': 0,
                    'total_keypoints': 0,
                    'avg_confidence': 0.0,
                    'max_confidence': 0.0,
                    'confidence_scores': np.array([])
                }
                all_scores.append(0.0)
                
        except Exception as e:
            print(f"\n❌ ERROR with {ref_name}:")
            print(f"   Error type: {type(e).__name__}")
            print(f"   Error message: {str(e)}")
            import traceback
            traceback.print_exc()
            raise e
    
    # Show debug plots
    if debug_plots:
        # Hide unused subplots
        for idx in range(plot_idx, len(axes)):
            axes[idx].axis('off')
        
        plt.tight_layout()
        plt.suptitle(f'ECDF Curves for Query: {Path(query_image_path).stem}\n(Higher AUC = Better Match, All curves end at (1,1))', 
                     fontsize=14, fontweight='bold', y=1.02)
        plt.show()
    
    processing_time = time.time() - start_time
    
    # Find best matches (Lowest AUC scores with your method)
    sorted_matches = sorted(match_results.items(), key=lambda x: x[1]['auc_score'], reverse=False)
    
    query_name = Path(query_image_path).stem
    # FInd the best match (lowest AUC)
    print("=" * 60)
    print(f"⏱️  Processing time: {processing_time:.2f} seconds")
    print(f"🏆 Top 3 matches (lowest AUC = best match):")
    for i, (name, data) in enumerate(sorted_matches[:-3]):
        print(f"   {i+1}. {name}: AUC = {data['auc_score']:.4f}")
    
    # Prepare results
    results = {
        'query_name': query_name,
        'best_match': sorted_matches[0][0],
        'best_score': sorted_matches[0][1]['auc_score'],
        'top_3_matches': sorted_matches[:3],
        'processing_time': processing_time,
        'total_comparisons': len(match_results)
    }
    
    return results

def compute_auc_from_scores_torch_exact(matching_scores):
    """
    Your EXACT working AUC computation method with proper (1,1) endpoint handling
    """
    valid = matching_scores[matching_scores > -1]
    if valid.numel() == 0:
        return 0.0

    sorted_scores = torch.sort(valid)[0]
    ecdf_y = torch.linspace(0, 1, steps=sorted_scores.numel(), device=sorted_scores.device)
    
    # Ensure we have the (1,1) endpoint for fair comparison
    if sorted_scores[-1] < 1.0:
        # Add the (1,1) point to complete the ECDF
        sorted_scores = torch.cat([sorted_scores, torch.tensor([1.0], device=sorted_scores.device)])
        ecdf_y = torch.cat([ecdf_y, torch.tensor([1.0], device=ecdf_y.device)])
    
    auc = torch.trapz(ecdf_y, sorted_scores).item()
    return auc
print("EXACT working turtle identification function ready!")

# Test with your problematic query
query_name = list(query_images.keys())[2]  # Should be "Michaelangelo R 18"
query_path = next(p for p in query_paths if p.stem == query_name)

results = identify_turtle_fixed(str(query_path), reference_library, extractor, matcher, device, debug_plots=True)

## 4. Run Turtle Identification

Let's identify our query turtles using the AUC-based methodology!

In [ ]:
# Process each query image with debug plots
identification_results = []
all_match_data = []

for query_name in list(query_images.keys()):
    # Reconstruct the full path
    query_path = next(p for p in query_paths if p.stem == query_name)
    
    results, match_data = identify_turtle_fixed(
        str(query_path),  # Pass the file path instead of the image array
        reference_library, 
        extractor, 
        matcher, 
        device,
        debug_plots=False  # ← Disable debug plotting
    )
    
    identification_results.append(results)
    all_match_data.append(match_data)

print(f"\n🎉 Identification complete for {len(identification_results)} query images!")

## 5. Visualization: Cumulative Distribution Plots

Let's visualize the confidence score distributions and AUC calculations for our best match.

In [ ]:
def plot_cumulative_distributions(match_data, top_n=5):
    """Plot cumulative distribution curves for the top N matches."""
    
    query_name = match_data['sorted_matches'][0][0].split()[0]  # Extract turtle name
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Plot 1: Cumulative distributions for top matches
    colors = plt.cm.Set1(np.linspace(0, 1, top_n))
    
    for i, (ref_name, data) in enumerate(match_data['sorted_matches'][:top_n]):
        if len(data['sorted_scores']) > 0:
            ax1.plot(data['sorted_scores'], data['cumulative_prob'], 
                    label=f"{ref_name} (AUC: {data['auc_score']:.3f})", 
                    linewidth=2, color=colors[i])
            
            # Fill area under curve for best match
            if i == 0:
                ax1.fill_between(data['sorted_scores'], data['cumulative_prob'], 
                               alpha=0.3, color=colors[i])
    
    ax1.set_xlabel('Confidence Score', fontweight='bold')
    ax1.set_ylabel('Cumulative Probability', fontweight='bold')
    ax1.set_title(f'Cumulative Distribution of Confidence Scores\nQuery: {query_name}', 
                  fontweight='bold')
    ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: AUC scores bar chart
    ref_names = [name for name, _ in match_data['sorted_matches'][:top_n]]
    auc_scores = [data['auc_score'] for _, data in match_data['sorted_matches'][:top_n]]
    
    bars = ax2.bar(range(len(ref_names)), auc_scores, 
                   color=colors[:len(ref_names)], alpha=0.7)
    
    # Highlight best match
    bars[0].set_color(colors[0])
    bars[0].set_alpha(1.0)
    bars[0].set_edgecolor('black')
    bars[0].set_linewidth(2)
    
    ax2.set_xlabel('Reference Images', fontweight='bold')
    ax2.set_ylabel('AUC Score', fontweight='bold')
    ax2.set_title('AUC Scores (Lower = Better Match)', fontweight='bold')
    ax2.set_xticks(range(len(ref_names)))
    ax2.set_xticklabels(ref_names, rotation=45, ha='right')
    
    # Add value labels on bars
    for i, (bar, score) in enumerate(zip(bars, auc_scores)):
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{score:.3f}', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()

# Plot for each query result
for i, match_data in enumerate(all_match_data):
    print(f"\n📊 Visualization for Query {i+1}:")
    plot_cumulative_distributions(match_data)

## 6. Results Summary

Let's create a comprehensive summary of our turtle identification results.

In [ ]:
# Create results summary
print("🐢 TURTLE IDENTIFICATION RESULTS SUMMARY")
print("=" * 70)

correct_identifications = 0
total_queries = len(identification_results)

for i, result in enumerate(identification_results):
    query_name = result['query_name']
    best_match = result['best_match']
    best_score = result['best_score']
    processing_time = result['processing_time']
    
    # Extract expected turtle name from query
    expected_turtle = query_name.split()[0]  # e.g., "Donatello" from "Donatello L 19"
    identified_turtle = best_match.split()[0]  # e.g., "Donatello" from "Donatello L"
    
    is_correct = expected_turtle == identified_turtle
    if is_correct:
        correct_identifications += 1
        status = "✅ CORRECT"
    else:
        status = "❌ INCORRECT"
    
    print(f"\nQuery {i+1}: {query_name}")
    print(f"  Expected: {expected_turtle}")
    print(f"  Identified: {identified_turtle} ({best_match})")
    print(f"  AUC Score: {best_score:.4f}")
    print(f"  Time: {processing_time:.2f}s")
    print(f"  Status: {status}")
    
    print(f"  Top 3 matches:")
    for j, (name, data) in enumerate(result['top_3_matches']):
        print(f"    {j+1}. {name}: {data['auc_score']:.4f}")

print("\n" + "=" * 70)
print(f"📈 OVERALL ACCURACY: {correct_identifications}/{total_queries} ({100*correct_identifications/total_queries:.1f}%)")
print(f"⏱️  AVERAGE PROCESSING TIME: {np.mean([r['processing_time'] for r in identification_results]):.2f}s")
print("=" * 70)

## 7. Performance Analysis

Let's analyze the performance characteristics of our AUC-based matching approach.

In [ ]:
# Performance analysis
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# 1. AUC Score Distribution
all_auc_scores = []
for match_data in all_match_data:
    for ref_name, data in match_data['match_results'].items():
        all_auc_scores.append(data['auc_score'])

ax1.hist(all_auc_scores, bins=20, alpha=0.7, edgecolor='black')
ax1.set_title('Distribution of AUC Scores', fontweight='bold')
ax1.set_xlabel('AUC Score')
ax1.set_ylabel('Frequency')
ax1.axvline(np.mean(all_auc_scores), color='red', linestyle='--', 
           label=f'Mean: {np.mean(all_auc_scores):.3f}')
ax1.legend()

# 2. Processing Time Analysis
processing_times = [r['processing_time'] for r in identification_results]
query_names = [r['query_name'] for r in identification_results]

bars = ax2.bar(range(len(processing_times)), processing_times, alpha=0.7)
ax2.set_title('Processing Time per Query', fontweight='bold')
ax2.set_xlabel('Query Image')
ax2.set_ylabel('Processing Time (seconds)')
ax2.set_xticks(range(len(query_names)))
ax2.set_xticklabels([name.split()[0] for name in query_names], rotation=45)

# Add value labels
for bar, time in zip(bars, processing_times):
    ax2.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.01,
            f'{time:.2f}s', ha='center', va='bottom', fontsize=9)

# 3. Number of Matches per Comparison
all_match_counts = []
match_labels = []
for match_data in all_match_data:
    for ref_name, data in match_data['match_results'].items():
        all_match_counts.append(data['num_matches'])
        match_labels.append(f"{match_data['sorted_matches'][0][0].split()[0]} vs {ref_name.split()[0]}")

ax3.scatter(range(len(all_match_counts)), all_match_counts, alpha=0.6)
ax3.set_title('Number of Keypoint Matches', fontweight='bold')
ax3.set_xlabel('Comparison Index')
ax3.set_ylabel('Number of Matches')
ax3.axhline(np.mean(all_match_counts), color='red', linestyle='--',
           label=f'Avg: {np.mean(all_match_counts):.1f}')
ax3.legend()

# 4. Accuracy by Turtle
turtle_accuracy = {}
for result in identification_results:
    expected = result['query_name'].split()[0]
    identified = result['best_match'].split()[0]
    
    if expected not in turtle_accuracy:
        turtle_accuracy[expected] = {'correct': 0, 'total': 0}
    
    turtle_accuracy[expected]['total'] += 1
    if expected == identified:
        turtle_accuracy[expected]['correct'] += 1

turtles = list(turtle_accuracy.keys())
accuracies = [turtle_accuracy[t]['correct']/turtle_accuracy[t]['total']*100 
              for t in turtles]

bars = ax4.bar(turtles, accuracies, alpha=0.7)
ax4.set_title('Identification Accuracy by Turtle', fontweight='bold')
ax4.set_xlabel('Turtle Name')
ax4.set_ylabel('Accuracy (%)')
ax4.set_ylim(0, 110)

# Add value labels
for bar, acc in zip(bars, accuracies):
    ax4.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 2,
            f'{acc:.1f}%', ha='center', va='bottom', fontweight='bold')

# Color code perfect accuracy
for bar, acc in zip(bars, accuracies):
    if acc == 100:
        bar.set_color('green')
        bar.set_alpha(0.8)

plt.tight_layout()
plt.suptitle('Performance Analysis: AUC-Based Turtle Identification', 
             fontsize=16, fontweight='bold', y=1.02)
plt.show()

## 8. Conclusions and Insights

### Key Findings:

1. **Novel AUC Methodology**: Our approach of using confidence score distributions and AUC calculation provides a robust matching metric that's less sensitive to outliers than simple confidence thresholding.

2. **Performance**: The system successfully identifies turtles with high accuracy, demonstrating the effectiveness of SIFT + LightGlue combination.

3. **Efficiency**: Processing times are reasonable for real-time applications, especially considering the comprehensive matching against all reference images.

### Technical Insights:

- **Lower AUC = Better Match**: Our methodology correctly identifies that cumulative distributions with lower AUC scores represent better feature correspondence
- **Robustness**: The approach handles varying image quality and lighting conditions well
- **Scalability**: The method scales linearly with the number of reference images

### Future Improvements:

1. **Optimization**: Cache feature extractions to improve processing speed
2. **Ensemble Methods**: Combine AUC scoring with other matching metrics
3. **Data Augmentation**: Expand reference library with additional poses and lighting conditions
4. **Real-time Processing**: Implement GPU optimization for faster inference

### Applications:

- **Wildlife Conservation**: Track individual sea turtles for population studies
- **Marine Biology Research**: Monitor turtle behavior and migration patterns  
- **Citizen Science**: Enable volunteers to contribute to turtle identification efforts

This demonstration showcases how computer vision and machine learning can contribute to marine conservation efforts!